In [ ]:
# cSpell: disable
# # A default setup cell.
# It imports environment variables, define 'devtools.debug" as a buildins, set PYTHONPATH, and code auto-reload
# Copy it in other Notebooks


from dotenv import load_dotenv
from rich import print

# type: ignore
load_dotenv(verbose=True)

%load_ext autoreload
%autoreload 2
%reset -f

In [ ]:
# First we initialize the model we want to use.
from typing import Literal

from genai_tk.core.llm_factory import get_llm
from langchain_core.tools import tool
from langgraph.checkpoint.memory import MemorySaver

# For this tutorial we will use custom tool that returns pre-defined values for weather in two cities (NYC & SF)


@tool
def get_weather(city: Literal["nyc", "sf"]):
    """Use this to get weather information."""
    if city == "nyc":
        return "It might be cloudy in nyc"
    elif city == "sf":
        return "It's always sunny in sf"
    else:
        raise AssertionError("Unknown city")


tools = [get_weather]
checkpointer = MemorySaver()
llm = get_llm()

### Custom React Agent (with Functional API) 

In [ ]:
from genai_blueprint.utils.langgraph import print_astream, print_stream
from genai_tk.extra.custom_react_agent import create_custom_react_agent

config = {"configurable": {"thread_id": 1}}

custom_agent = create_custom_react_agent(llm, tools, checkpointer)
result0 = await custom_agent.ainvoke("What's the weather in NYC?", config)
print(result0)

In [ ]:
stream = custom_agent.stream("What's it known for?", config)
print_stream(stream)

In [ ]:
# Test Memory
stream = custom_agent.astream("What's the main Landmarks ?", config)
await print_astream(stream)


### Same with prebuilt react agent